In [1]:
import pickle
import nltk
import json
from tqdm import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer

APP_FOLDER = '../app/data/processed/'

In [2]:
with open(APP_FOLDER + 'dish2rest.plk', 'rb') as f:
    dish_info = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../app/data/processed/dish2rest.plk'

In [ ]:
with open(APP_FOLDER + 'restinfo.plk', 'rb') as f:
    restinfo = pickle.load(f)

In [4]:
with open('./categories/Chinese.pkl', 'rb') as f:
    cuisine_reviews = pickle.load(f)    
with open('ChineseDishes.txt', 'r') as f:
    dish_names = [line.replace("\n", "").lower() for line in f.readlines()]

for _review in tqdm(cuisine_reviews):
    _review['sentences'] = nltk.sent_tokenize(_review['text'])

sia = SentimentIntensityAnalyzer()

dish2rest = {}
for _name in tqdm(dish_names):
    _review_count = 0
    _sum_review_polarity = 0
    _sum_rest_polarity = 0
    
    for _review in cuisine_reviews:
        _rest_id = _review['business_id']
        _review_id = _review['review_id']
        _review_polarity = 0
        _review_sentence_count = 0
        for _sentence in _review['sentences']:
            _count = _sentence.count(_name)
            if _count > 0:
                _review_sentence_count += 1
                _ss = sia.polarity_scores(_sentence)
                _score = (_ss['pos'] + 1e-10)/ (_ss['neg'] + 1e-10) * 0.5
                if _score <= 1:
                    _review_polarity += _score
                else:
                    _review_polarity = 1
        
        if _review_sentence_count > 0:
            _review_polarity = _review_polarity / _review_sentence_count
            _sum_review_polarity += _review_polarity
            _review_count += 1
            if not(_name in dish2rest):
                dish2rest[_name] = {'rest2review_count':{}, 
                                     'review2polarity':{}, 
                                     'rest2polarity':{}, 
                                     'rest2score':{},
                                     'review_count': 0,
                                     'rest_count': 0}
                    
            dish2rest[_name]['review2polarity'][_review_id] = _review_polarity
            if _rest_id in dish2rest[_name]['rest2review_count']:
                dish2rest[_name]['rest2review_count'][_rest_id] += 1
                dish2rest[_name]['rest2polarity'][_rest_id] += _review_polarity
            else:
                dish2rest[_name]['rest2review_count'][_rest_id] = 1
                dish2rest[_name]['rest2polarity'][_rest_id] = _review_polarity
            
    if _name in dish2rest:
        for _id in dish2rest[_name]['rest2polarity']:
            dish2rest[_name]['rest2score'][_id] = dish2rest[_name]['rest2polarity'][_id]
            dish2rest[_name]['rest2polarity'][_id] /= dish2rest[_name]['rest2review_count'][_id]
            _sum_rest_polarity += dish2rest[_name]['rest2polarity'][_id]

        dish2rest[_name]['rest2polarity'] = sorted(dish2rest[_name]['rest2polarity'].items(), key=lambda kv: kv[1], reverse = True)
        dish2rest[_name]['rest2review_count'] = sorted(dish2rest[_name]['rest2review_count'].items(), key=lambda kv: kv[1], reverse = True)
        dish2rest[_name]['rest2score'] = sorted(dish2rest[_name]['rest2score'].items(), key=lambda kv: kv[1], reverse = True)
        dish2rest[_name]['review_count'] = len(dish2rest[_name]['review2polarity'])
        dish2rest[_name]['rest_count'] = len(dish2rest[_name]['rest2review_count'])
        dish2rest[_name]['rest_polarity'] = _sum_rest_polarity / dish2rest[_name]['rest_count']
        dish2rest[_name]['review_polarity'] = _sum_review_polarity / dish2rest[_name]['review_count']

sorted_rest_dishes = sorted(dish2rest.items(), key=lambda kv: kv[1]['rest_count'], reverse=True)
sorted_review_dishes = sorted(dish2rest.items(), key=lambda kv: kv[1]['review_count'], reverse=True)

with open (APP_FOLDER + 'dish2rest.plk', 'wb') as f:
    pickle.dump(sorted_rest_dishes, f)

100%|██████████| 196/196 [00:40<00:00,  6.64it/s]
